# Probability for Rendering

Hi! Welcome to Computer Graphics at ShanghaiTech University!
In this lab, we'll guide you through some simple yet important factors in building a _Physically-based Render_.
It might also serve as an excuse for you to review probability theory.

Please execute the following code section to make sure that all dependencies are installed.


In [ ]:
%pip install numpy
%pip install pandas
%pip install scipy
%pip install matplotlib
%pip install seaborn

In [6]:
import numpy as np
import pandas as pd
import statistics
import math
import random

import scipy.integrate as integrate
import scipy.special as special
import scipy.optimize as optimize
from scipy.stats import norm, truncnorm
import matplotlib.pyplot as plt
import seaborn

np.random.seed(42)
seaborn.set_theme()

## Estimator Basics

You might have heard of the words _biased_, _consistent_ or _unbiased_ along the way of learning probability.
Generally, these words are used to describe _estimators_.
**Estimators are random variables**. The reason they are called _estimators_ is that, they are used to estimate some _underlying values_.
So for arbitrary _estimator_, we can take, for example, expectation and variance on them.
Don't worry if you don't understand what do me mean by _underlying values_, by the end of this lab, you'll get a comprehensive understanding of estimator.

Here is a simple and incomplete abstraction of normal distribution $\mathcal{N}(\mu_i, \sigma_i^2)$.


In [7]:
class Normal:
    def __init__(self, n: int = 100) -> None:
        self.n_ = n  # num samples
        self.mu_ = math.sqrt(42)
        self.sigma_ = 24
        self.samples_ = np.random.normal(self.mu_, self.sigma_, self.n_)

    def sample(self) -> float:
        self.n_ -= 1
        result = self.samples_[self.n_]
        return result

Suppose that we're in a new universe with unknown light speed $c$, each sample (experiment) of the previous class will give us a _measure_ of light speed.
You, our explorer, have limited resource to perform experiments, that you can only perform experiments for $100$ times.

But you are asked by your commander to give the prediction of $c^2$, which is unexpectingly important in scientific computing.
Your rough plan is to design an estimator $Y$ that can give you the prediction, i.e., $\mathbb{E}\left[Y\right] = c^2$.
Your commander told you that, _all estimator design begins with writing this form of formula_.

After some rough thinking, you think there are three approaches to design the estimator.
And you acknowledged that calling `sample` for $n$ times is equivalent to producing $n$ i.i.d. random variables $X_i$ where $\mathbb{E}\left[X_i\right] = c$.
For now, you don't have to understand what _biased_, etc. really means.

$$
\left\{
\begin{aligned}
  Y_1 &= \dfrac{1}{100}\sum_{i = 1}^{100}{X_i^2} &\text{(biased)} \\
  Y_2 &= \left(\dfrac{1}{100}\sum_{i = 1}^{100}{X_i}\right)^2 &\text{(consistent)} \\
  Y_3 &= \left(\dfrac{1}{50}\sum_{i = 1}^{50}{X_i}\right) \times \left(\dfrac{1}{50}\sum_{i = 51}^{100}{X_i}\right) &\text{(unbiased)} \\
\end{aligned}
\right.
$$


As an appetizer, your task is to fill the following Python functions with the formulas, which will give us the estimation of $c^2$.


In [8]:
def biased(normal: Normal) -> float:
    result = []
    # TODO: your implementation of the *biased* estimator above
    for _ in range(100):
        result.append(math.pow(normal.sample(), 2))
    return statistics.mean(result)


def consistent(normal: Normal) -> float:
    result = []
    # TODO: your implementation of the *consistent* estimator above
    for _ in range(100):
        result.append(normal.sample())
    return math.pow(statistics.mean(result), 2)


def unbiased(normal: Normal) -> float:
    # TODO: your implementation of the *unbiased* estimator above
    # First estimator
    result = []
    for _ in range(50):
        result.append(normal.sample())
    I1 = statistics.mean(result)

    # Second estimator
    result = []
    for _ in range(50):
        result.append(normal.sample())
    I2 = statistics.mean(result)

    return I1 * I2


Great! As a numerical verification, execute the following code,


In [ ]:
# A sample-based mean/variance estimator
def test(func, n: int = 1000) -> float:
    result = []
    for _ in range(n):
        result.append(func())
    return (statistics.mean(result), statistics.variance(result))


pd.options.display.float_format = "{:.2f}".format
df = pd.DataFrame(
    index=["biased", "consistent", "unbiased"], columns=["mean", "variance"]
)
df.loc["biased"] = test(lambda: biased(normal=Normal()))
df.loc["consistent"] = test(lambda: consistent(normal=Normal()))
df.loc["unbiased"] = test(lambda: unbiased(normal=Normal()))
print(df)


The performance discrepancy is obvious since the ground truth of $c^2$ is $42$, only the _unbiased_ estimator approaches the result.

But why is that?

Sure, let's take their expectation respectively to observe the result. Recall that,

$$
  \dfrac{1}{n}\sum_{i = 1}^{n} \mathcal{N}(\mu_i, \sigma_i^2) = \mathcal{N}\left(\dfrac{1}{n}\sum_{i = 1}^{n}{\mu_i}, \dfrac{1}{n^2}\sum_{i = 1}^{n}{\sigma_i^2}\right)
$$

and

$$
  \mathbb{V}\left[X\right] = \mathbb{E}\left[X^2\right] - \mathbb{E}^2\left[X\right]
$$

Then let's test our estimators,

$$
\begin{aligned}
  \mathbb{E}\left[Y_1\right] &= \dfrac{1}{100}\sum_{i = 1}^{100}{\mathbb{E}\left[X_i^2\right]} \\
                             &= \dfrac{1}{100}\sum_{i = 1}^{100}{\left(\mathbb{V}\left[X_i\right] + \mathbb{E}^2\left[X_i\right]\right)} \\
                             &=  c^2 + \dfrac{1}{100}\sum_{i = 1}^{100}{\mathbb{V}\left[X_i\right]} \approx 576 + 41 = 617
\end{aligned}
$$


At this point, I believe you have understood what _biased estimator_ means, which is, $\mathbb{E}\left[{Y_i}\right] - c^2 > 0$, and no matter how many more samples we obtain, the bias will be pertained, which is the variance of all samples.

We could introduce the term _consistent estimator_ now. **Asymptotically**, when the number of samples we can obtain converges to $+\infty$, $\mathbb{E}\left[Y_i\right] = c^2$. Here is the example,
Let $X = \sum_{i} X_i = \mathcal{N}\left(\overline{\mu}, \overline{\sigma^2}/n\right)$,

$$
  \mathbb{E}\left[Y_2\right] = \mathbb{E}\left[X^2\right] = \mathbb{V}\left[X\right] + \mathbb{E}^2\left[X\right] = c^2 + \dfrac{\sigma^2}{n}
$$

Then when $n \rightarrow +\infty$, the biased is eliminated, which corresponds to the definition of _consistent estimator_.
However, whatever the parameter `n` of the `test` function is (not the `n` in `class Normal`, there is a crucial difference between these two $n$, why?), we will not obtain the correct result, since for a specific `n`, the estimator is biased at all.

Consistent estimator is great! But we deserve somehow more. For the unbiased estimator, $\forall n \ge 1, \mathbb{E}\left[Y_3\right] = c^2$. The proof is yet easy,

$$
  \mathbb{E}\left[Y_3\right] = \mathbb{E}\left[I_1\right] \times \mathbb{E}\left[I_2\right] = c^2.
$$

where $I_1$ and $I_2$ are the independent r.v.s generated from disjoint ranges of $X_i$.

Before the end of this section, there is one thing that worth mentioning. _Biased_ do not necessarily imply that the estimator is inefficient.
Although the previous example's bias is due to inadvertent wrong design.
In real cases, sometimes bias are introduced intentionally to obtain low variance,
sometimes bias are difficult to mitigate, or it takes a lot of effort to eliminate, so people just leave it as it is.
Anyway, you'll meet a lot of biased but efficient estimators along the way, don't panic.


### Conclusion and Exercise

Here's a simple conclusion. Strategy makes a difference. At this point, you should at least understand what are estimators, what are _biased_, _consistent_ and _unbiased_ estimators.
As for designing estimators, which is an extremely difficult and interesting field (not just) in Computer Graphics, I'll present a more interesting example here.


Your commander is very satisfied with your estimation, while traveling through a patch of interstellar dust, he asked you to give another _unbiased_ estimation to $\exp(c)$, i.e., $e^c$.
Unlike the last time, this time you have more resources to conduct more experiments.

In introducing the techniques to solve this problem, we need to retrospect some basics.

Suppose we have an estimator (r.v.) $X$. Recall that $X$ is a (measurable) map. Yes, random variables are not variables, they are maps.
And there can have a _distribution_ defined on its _range_. Since $X$ is a map, and maps can be composed, i.e., $f \circ g$.
Any _valid_ functions applied on $X$ can form a new random variable.
For example, $f(X)$ is yet another random variable.
Further, $X \cdot Y$ is a new r.v., $p(X)$ is yet r.v., where $p$ is its distribution function.

Then, our estimator design for $\exp(c)$ is $X$, where $\mathbb{E}\left[X\right] = \exp(c)$. Then, perform Taylor series expansion on $\exp(c)$.

$$
  \exp(c) = 1 + \dfrac{c}{1!} + \dfrac{c^2}{2!} + \dfrac{c^2}{3!} + \cdots \; \text{(countable series)}
$$

$X$ can be decomposed like this, $\mathbb{E}\left[X\right] = 1 + \sum_{i = 1}^{\infty}{\mathbb{E}\left[X_i\right]}$.
So our task, actually, is to evaluate this infinite series. But we don't have infinitely many resources to use! Here is where the trick comes.


We let $\mathbb{E}\left[X_i\right] = \mathbb{E}\left[I_i \cdot X_i'\right]$,
where $I_i$ is an indicator variable controlling whether this term $X_i$ should be taken into the summation.
Since $\mathbb{E}\left[I_i \cdot X_i'\right] = 1 \cdot X_i' \cdot P(I_i = 1)$, $X_i' = X_i P(I_i)^{-1}$.
**For any** series of indicator variables $\{I_i\}_{i = 1}^{+\infty}$, this infinite series will evaluate to a correct result.

We somehow expect that the series' evaluation process can be terminated in some $i$, then let $K \sim \mathrm{Geo}(p)$, and $I_i = [K > i]$, written in [Iverson bracket](https://www.wikiwand.com/en/Iverson%20bracket), we obtain the unbiased estimator of $\exp(c)$, which is

$$
\begin{aligned}
  \mathbb{E}\left[X\right] &= \exp(c) \\
    &= 1 + \sum_{i = 1}^{+\infty}{I_i \dfrac{X_i}{P(I_i = 1)}}
    = 1 + \sum_{i = 1}^{K-1}{\dfrac{X_i}{P(K > i)}}
\end{aligned}
$$

where $\mathbb{E}\left[X_i\right] = \dfrac{c^i}{i!}$, whose unbiased estimator design is known from our previous exercise, and $P(K > i) = (1 - p)^i, \forall i \ge 1$.

Further, one way to obtain (sample) $K$ is to actually simulate the process, where _success_ means _termination_. Try to implement it here!


In [ ]:
class Normal:
    def __init__(self, mu: float) -> None:
        self.mu_ = mu
        self.sigma_ = 1.0

    def sample(self) -> float:
        return np.random.normal(self.mu_, self.sigma_)


def rr(func) -> float:
    p = 0.1  # terminate with a prob 0.1
    result = 1.0  # 1 + ... (in Taylor series)
    i = 0
    while True:
        # TODO: fill in your implementation here
        i += 1
        if random.random() <= p:
            break
        result += func(i) / math.pow((1 - p), i)
    return result


def exp_series(func, i: int) -> float:
    # TODO: fill in your implementation here
    result = 1.0
    for j in range(i):
        result *= func() / (j + 1)
    return result


mu = 2.0
print(
    f"the estimation of [exp({mu})={math.exp(mu):.4f}] is",
    test(lambda: rr(lambda i: exp_series(func=Normal(mu).sample, i=i)))[0],
)

This approach is called [Russian Roulette](https://www.wikiwand.com/en/Russian_roulette), abbreviate to RR.
RR is a such powerful tool to construct estimator on infinite series that you'll see its application later in your rendering homework.

In conclusion, we've constructed an unbiased estimator to estimate the form like $\exp(c)$ with RR.
Note that we already have an unbiased estimator for $\forall i \ge 1, c^i$.
$c$ is not necessarily a constant. We'll see an example later.


## Monte Carlo Integration

Monte Carlo Integration is a realization of _estimator design_ and _numerical integration_, which designs estimator on a broad range of integral.
To understand how powerful the Monte Carlo Integration is, we start with the definition of _numerical integration_.

Unlike our homework helper (_Mathematica_) in the mathematical analysis course, which is symbolic integration, _numerical integration_ intends to "calculate the numerical value of a definite integral".
In 1-D case, we might want,

$$
  c = \int_{a}^{b}{f(x) \, \mathrm{d} x},
$$

which can be abstracted to $I_{\mathrm{numerical}}\left[\mathbb{R}, [a, b] \subset \mathbb{R}, f\right] \rightarrow \mathbb{R}$, where $f$ might at least defined on $[a, b]$.
Note that this function $I_{\mathrm{numerical}}$ accepts several things, a _set_, the _integral domain_ and a _real-valued function_ on this domain.

A common implementation of $I_{\mathrm{numerical}}$ is to partition $[a, b]$ into disjoint _subintervals_, then perform some kinds of summations.
We'll neglect the detail here because Monte Carlo Integration is somehow completely different.
Those who are interested can refer to the explanation of numerical integration on [Wikipedia](https://www.wikiwand.com/en/Numerical_integration).

To further introduce Monte Carlo Integration to you, you need to understand why we need it.
It is what motivation that drives us away from these simple and intuitive numerical integration approaches?
You will be amazed by its _power_, _elegance_, and _simplicity_.


Again, Monte Carlo Integration is the realization of _estimator design_. You should ask two questions,

- What is the underlying value?
- How can we design the estimator?

The underlying value, to simplify, is a definite integral where $c = \int_{a}^{b}{f(x) \, \mathrm{d}x}$ (actually, to apply Monte Carlo Integration to Rendering, readers might need to understand integration on general measure space).
Recall that we expect an estimator $Y$ such that,

$$
\begin{aligned}
  \mathbb{E}\left[Y\right] &= c = \int_{a}^{b}{f(x) \, \mathrm{d}x} \\
    &= \int_{a'}^{b'}{ y \cdot p(y) \, \mathrm{d}y} \\
\end{aligned}
$$

to align these two integrals, $a' = a, b' = b$, most importantly,

$$
  Y = \dfrac{f(X)}{p(X)} = g(X).
$$

where $X$ can be **any** valid r.v. To verify,

$$
  \mathbb{E}\left[g(X)\right] = \int_{a'}^{b'}\dfrac{f(x)}{\cancel{p(x)}} \cdot \cancel{p(x)} \, \mathrm{d}x.
$$

Isn't it too simple? But it's this simple method that builds the field _Rendering_.

Let's demonstrate this process with code.


You and your fleet are moving through a known stretch of interstellar dust, again!
You can your commander realized that you need to perform integration on the density of dust along the way you are traveling to estimate and reduce energy consumption, which is an integral like this,

$$
  c = \int_{0}^{4.5}{\sigma(t) \, \mathrm{d}t},
$$


Random variables from now on are abstracted into a class, which is supposed to be easy to understand.
We've provided three examples here, note that `Normal` distribution should not be used as the preceding random variable of Monte Carlo estimator,
since it is not bounded, otherwise the estimator is biased.

Estimators are generated by factory functions like `make_monte_carlo_estimator`.


In [ ]:
# https://docs.scipy.org/doc/scipy/tutorial/integrate.html
# Use this simple but non-trivial function
def func(x):
    return special.jv(2.5, x)


class RandomVariable:
    """
    Definition of RandomVariable
        Not a formal definition, but straight-forward
    """

    def __init__(self):
        pass

    def sample(self) -> float:
        assert False

    def pdf(self, x: float) -> float:
        assert False

    def expect(self, n: int = 1000) -> float:
        # LLN
        result = 0.0
        for _ in range(n):
            result += self.sample()
        return result / n


class Uniform(RandomVariable):
    def __init__(self, a: float, b: float):
        self.a_ = a
        self.b_ = b
        self.inv_length_ = 1 / (b - a)

    def sample(self) -> float:
        return np.random.uniform(low=self.a_, high=self.b_)

    def pdf(self, x: float) -> float:
        return self.inv_length_


class Normal(RandomVariable):
    def __init__(self, mu, sigma):
        self.mu_ = mu
        self.sigma_ = sigma
        self.pdf_ = lambda x: norm.pdf(x, self.mu_, self.sigma_)

    def sample(self) -> float:
        return np.random.normal(self.mu_, self.sigma_)

    def pdf(self, x: float) -> float:
        return self.pdf_(x)


class TruncNormal(RandomVariable):
    def __init__(self, a: float, b: float):
        self.a_ = a
        self.b_ = b
        self.mu_ = (a + b) / 2
        self.sigma_ = b - a
        self.cdf_ = lambda x: norm.cdf(x, self.mu_, self.sigma_)
        self.pdf_ = lambda x: norm.pdf(x, self.mu_, self.sigma_)
        self.factor_ = self.cdf_(b) - self.cdf_(a)

    def sample(self) -> float:
        # rejection sampling
        sample = np.random.normal(self.mu_, self.sigma_)
        while not (self.a_ <= sample <= self.b_):
            sample = np.random.normal(self.mu_, self.sigma_)
        return sample

    def pdf(self, x: float) -> float:
        return self.pdf_(x) / self.factor_


class NumericalIntegral:
    """
    An integral can be abstracted to
        1. A function defined on Omega
        2. A (measurable) set (indicated by low, high)
    """

    def __init__(self, func, low, high) -> None:
        self.func_ = func
        self.low_ = low
        self.high_ = high
        self.max_ = None
        self.min_ = None

    def get_value(self):
        # Use general numerical integration method
        assert isinstance(self.low_, float) and isinstance(self.high_, float)
        return integrate.quad(self.func_, self.low_, self.high_)[0]

    def get_max(self):
        # Reserved for MCMC methods
        assert isinstance(self.low_, float) and isinstance(self.high_, float)
        # https://stackoverflow.com/questions/10146924/finding-the-maximum-of-a-function
        if self.max_ is None:
            self.max_ = -optimize.minimize_scalar(
                lambda x: -self.func_(x),
                bounds=[self.low_, self.high_],
                method="bounded",
            ).fun
        return self.max_

    def get_min(self):
        # Reserved for residual-tracking
        assert isinstance(self.low_, float) and isinstance(self.high_, float)
        # https://stackoverflow.com/questions/10146924/finding-the-maximum-of-a-function
        if self.min_ is None:
            self.min_ = optimize.minimize_scalar(
                self.func_, bounds=[self.low_, self.high_], method="bounded"
            ).fun
        return self.min_

    def make_monte_carlo_estimator(self, rv_factory) -> RandomVariable:
        class Estimator(RandomVariable):
            def __init__(self_):
                self_.base_rv_ = rv_factory(self.low_, self.high_)

            def sample(self_) -> float:
                # TODO: fill in your implementation here
                spl = self_.base_rv_.sample()
                return self.func_(spl) / self_.base_rv_.pdf(spl)

        return Estimator()


integral = NumericalIntegral(func=func, low=0.0, high=4.5)
est_uniform = integral.make_monte_carlo_estimator(Uniform)
est_tnormal = integral.make_monte_carlo_estimator(TruncNormal)
print(f"groundtruth: {integral.get_value():.4f}")
print(f"uniform:     {est_uniform.expect():.4f}")
print(f"tnormal:     {est_tnormal.expect():.4f}")


Hey! But things are not finished yet. Monte Carlo _Estimators_ are simple, this is what we know. AND, one more thing,

**Monte Carlo Integration can be performed on arbitrarily high dimension.**

Repeat it for three times, Monte Carlo Integration can be performed on arbitrarily high dimension,
Monte Carlo Integration can be performed on arbitrarily high dimension,
Monte Carlo Integration can be performed on arbitrarily high dimension.

Reflected into the code, this says, the set in $\mathbb{R}$ indicated by `low` and `high` can be switched into some unknown representation.

To avoid spoilers on one of the most the interesting part of Computer Graphics, we'll not delve into this topic.
Later, we'll have the chance to utilize this awesome method to imitate the real-world magic in our personal computers!


### Unbiased Variance Estimator

To test your understanding of the above,
we designed a simple exercise involving both Monte Carlo Integration and unbiased estimator design.

Here it is! Design an **unbiased** variance estimator and implement it!
To be specific, we asked you to design an estimator $\theta$ for a random variable $X$,
so that $\mathbb{E}\left[\theta\right] = \mathrm{Var}(X)$.

You are given $100$ i.i.d. samples of $X$, $\left\{X_i\right\}_{i=1}^{100}$.

Hint:

$$
\begin{aligned}
  \mathrm{Var}(X)
  & = \mathbb{E}\left[X^2\right] - \mathbb{E}^2\left[X\right] \\
  & = \int_{-\infty}^{\infty}{x^2 p(x) \, \mathrm{d} x} - \mathbb{E}^2\left[X\right].
\end{aligned}
$$

Answer:
Design $\theta$ so that (assume $n$ is even)

$$
  \theta = \left[\dfrac{1}{n} \sum_{i=1}^{n}X_i^2\right] -
  \left[
    \left(\dfrac{2}{n}\sum_{i=1,3,\cdots}^{n}X_i\right)
    \left(\dfrac{2}{n}\sum_{i=2,4,\cdots}^{n}X_i\right)
  \right]
$$


In [ ]:
def make_variance_estimator(rv: RandomVariable, n: int = 100) -> RandomVariable:
    class Estimator(RandomVariable):
        def __init__(self, n) -> None:
            super().__init__()
            self.n_ = n

        def sample(self) -> float:
            X = [rv.sample() for _ in range(self.n_)]
            X = np.array(X)
            return 1 / self.n_ * np.sum(X * X) - (
                2 / self.n_ * np.sum(X[: (self.n_ // 2)])
            ) * (2 / self.n_ * np.sum(X[(self.n_ // 2) :]))

    return Estimator(n)


print("normal calculated:", make_variance_estimator(Normal(42, 4)).expect())
print("       expected:", 16)
print("uniform calculated:", make_variance_estimator(Uniform(0, 1)).expect())
print("        expected", 1 / 12.0)


### Combine them together

This section serves as an appendix, to combine the previously mentioned methods together, to estimate this!

$$
  \mathrm{Tr}(p \rightarrow p') = \exp\left(-\int_{0}^{t}{\sigma(t) \, \mathrm{d} t}\right)
$$

Which is the transmittance between two points. We might expect an _unbiased_ estimator on this.
This formula is common in _Volume Rendering_, designing an efficient unbiased estimator of which is generally regarded as a hard task.

But, we already have the necessary tools to accomplish this task, just combine the previous functions!


In [ ]:
def make_taylor_transmittance_estimator(integral: NumericalIntegral):
    estimator = integral.make_monte_carlo_estimator(Uniform)

    class Estimator(RandomVariable):
        def __init__(self) -> None:
            super().__init__()

        def sample(self) -> float:
            return rr(lambda i: exp_series(func=estimator.sample, i=i))

    return Estimator()


m_integral = NumericalIntegral(func=lambda x: -func(x), low=0.0, high=4.5)
tr_est_uniform = make_taylor_transmittance_estimator(m_integral)
print(f"groundtruth: {np.exp(m_integral.get_value()):.4f}")
print(f"mean:        {tr_est_uniform.expect():.4f}")
print(f"variance:    {make_variance_estimator(tr_est_uniform, 10).expect():.4f}")


### MCMC Methods

This part is completely optional, but we recommend you to execute it and check its result comparing to previous method.
It is based on [This paper](https://cs.dartmouth.edu/wjarosz/publications/novak14residual.pdf),
you can refer to it for pseudo code.
It only demonstrates the possibility of highly efficient method for the previous formula.

Execute this code! Experience the power of the MCMC methods. You don't have to understand it for now.


In [ ]:
def delta_tracking(integral: NumericalIntegral, max_f: float):
    x = integral.low_
    while True:
        eta = np.random.uniform()
        x -= np.log(1 - eta) / max_f
        if x >= integral.high_:
            break
        epsilon = np.random.uniform()
        if epsilon <= integral.func_(x) / max_f:
            break
    return x


def make_delta_tracking_transmittance_estimator(
    integral: NumericalIntegral,
) -> RandomVariable:
    max_f = integral.get_max()

    class Estimator(RandomVariable):
        def __init__(self) -> None:
            super().__init__()

        def sample(self) -> float:
            return 1 if delta_tracking(integral, max_f) > integral.high_ else 0

    return Estimator()


def make_ratio_tracking_transmittance_estimator(
    integral: NumericalIntegral,
) -> RandomVariable:
    max_f = integral.get_max()

    class Estimator(RandomVariable):
        def __init__(self) -> None:
            super().__init__()

        def sample(self) -> float:
            x = integral.low_
            T = 1
            while True:
                eta = np.random.uniform()
                x -= np.log(1 - eta) / max_f
                if x >= integral.high_:
                    break
                T *= 1 - integral.func_(x) / max_f
            return T

    return Estimator()


def make_residual_tracking_transmittance_estimator(
    integral: NumericalIntegral, f_c: float
) -> RandomVariable:
    class Estimator(RandomVariable):
        def __init__(self, f_c) -> None:
            super().__init__()
            self.f_c_ = f_c
            self.f_r_ = -optimize.minimize_scalar(
                lambda x: -np.abs(integral.func_(x) - f_c),
                bounds=[integral.low_, integral.high_],
                method="bounded",
            ).fun

        def sample(self) -> float:
            x = integral.low_
            Tc = np.exp(-self.f_c_ * (integral.high_ - integral.low_))
            Tr = 1.0
            while True:
                eta = np.random.uniform()
                x -= np.log(1 - eta) / self.f_r_
                if x >= integral.high_:
                    break
                Tr *= 1 - (integral.func_(x) - self.f_c_) / self.f_r_
            return Tc * Tr

    return Estimator(f_c)


tr_est_dt = make_delta_tracking_transmittance_estimator(integral)
tr_est_rt = make_ratio_tracking_transmittance_estimator(integral)
tr_est_rst_mean = make_residual_tracking_transmittance_estimator(
    integral, integral.get_value() / (integral.high_ - integral.low_)
)
print(f"groundtruth:  {np.exp(m_integral.get_value()):.4f}")
print(f"our mean:     {tr_est_uniform.expect():.4f}")
print(f"dt  mean:     {tr_est_dt.expect():.4f}")
print(f"rt  mean:     {tr_est_rt.expect():.4f}")
print(f"rst mean:     {tr_est_rst_mean.expect():.4f}")
print(f"our variance: {make_variance_estimator(tr_est_uniform, 10).expect():.4f}")
print(f"dt  variance: {make_variance_estimator(tr_est_dt, 10).expect():.4f}")
print(f"rt  variance: {make_variance_estimator(tr_est_rt, 10).expect():.4f}")
print(f"rst variance: {make_variance_estimator(tr_est_rst_mean, 10).expect():.4f}")


Their implementations seem easy. Well, yes!
But the derivation is rather complex, and requires you to manage the previous concepts.
You can refer to [this material](https://cs.dartmouth.edu/wjarosz/publications/novak14residual-supplemental1.pdf) for more information.
We'll only prove _Ratio-Tracking_ here for you to entertain ;)

Happy grinding, and welcome to Computer Graphics!


## Multiple Importance Sampling

Remember _Monte Carlo Integration_ that we discussed previously? Consider a predetermined function to integrate, say $f(x) = x^2$, and put aside the fact that we can analytically integrate this function. Is there a special choice of random variable that would yield a more efficient estimator for the integral of $f$? Intuitively, we might guess that when $p(X) \sim f$, we achieve the best estimator. This is indeed true, because $f(X) / p(X)$ becomes a constant. Let's explore this by checking it with some code.


In [ ]:
class SquareImportanceSampler(RandomVariable):
    def __init__(self, a: float, b: float):
        self.a_ = a
        self.b_ = b
        self.integral_ = 1 / 3 * (b**3 - a**3)

    def sample(self) -> float:
        u = np.random.uniform(low=0.0, high=1.0)
        return (self.a_**3 + u * 3 * self.integral_) ** (1 / 3)

    def pdf(self, x: float) -> float:
        return x**2 / self.integral_


def print_sampler_report(func, sampler):
    integral = NumericalIntegral(func=func, low=0.1, high=1.1)
    est_uniform = integral.make_monte_carlo_estimator(Uniform)
    est_importance = integral.make_monte_carlo_estimator(sampler)
    print(f"est_uniform:       {est_uniform.expect():.4f}")
    print(f"est_yours:         {est_importance.expect():.4f}")
    print(f"variance_uniform:  {make_variance_estimator(est_uniform, 10).expect():.4f}")
    print(
        f"variance_yours:    {make_variance_estimator(est_importance, 10).expect():.4f}"
    )


print_sampler_report(lambda x: x**2, SquareImportanceSampler)

As the results show, the importance sampler provides a perfect estimator with an accurate mean and zero variance. But is this technique generally applicable? Regrettably, the answer is no. The problem arises in two main aspects:

- Typically, we cannot derive the perfect sampler because the underlying function $f$ is either unavailable or cannot be integrated and inverted. In rendering, we often design estimators that are _good enough_ but not perfect.
- We need to use the same estimator for different $f$ functions! That's the key point. Especially in rendering, we usually have only one type of estimator, but the scene configuration can vary greatly. Even if we design an estimator that _works well_ for a specific scene, it is challenging to generalize it to different scenes.

Let's try it out by creating a different function to integrate.


In [ ]:
print_sampler_report(lambda x: 1 / x, SquareImportanceSampler)

Well, the importance sampler gives a much worse result than even the baseline. If you are designing a library to integrate user input, this is definitely not what you want: the input can come in any form, and being good at only a single function achieves nothing.

Introducing _Multiple Importance Sampling_ (MIS), one of the **most powerful** techniques in rendering. Many scenes cannot be effectively rendered without MIS. Bidirectional Path Tracing relies entirely on MIS, serving as a method to construct _efficient samplers_ for specific cases. However, the idea behind which is surprinsingly simple, and can be given in a few lines.

We make the statement clear. We are to design a single estimator. For different inputs, we construct different efficient (sub-)samplers, say, $X_1$ for $f_1$ and $X_2$ for $f_2$. The function to be integrated is randomly chosen from either $f_1$ or $f_2$ but you don't know which one is chosen. We first combine $X_1$ and $X_2$ linearly.

$$
  X = c_1 X_1 + c_2 X_2,
$$

where $c_1 + c_2 = 1$. We have

$$
\begin{aligned}
  \mathbb{E}[X] & = c_1 \mathbb{E}[X_1] + c_2 \mathbb{E}[X_2] = I,
\end{aligned}
$$

where $I$ is the result of the integral on $f$. This seems trivially correct, but one crucial observation appears when we expand the expectation into integral:

$$
\begin{aligned}
  c_1 \mathbb{E}[X_1] + c_2 \mathbb{E}[X_2]
  & = \int_\Omega c_1 \dfrac{f(x)}{p_1(x)} p_1(x) \, \mathrm{d} x + \int_\Omega c_1 \dfrac{f(x)}{p_2(x)} p_2(x) \, \mathrm{d} x \\
  & = \int_{\Omega} (c_1 + c_2) f(x) \, \mathrm{d} x,
\end{aligned}
$$

where $p_1(x)$ and $p_2(x)$ are the PDF of the random variable respectively. See where? As long as $c_1 + c_2 = 1$, the equation holds. So why not extending $c_1$ and $c_2$ to a function on $X$, to $c_1(x)$ and $c_2(x)$. This gives

$$
  \mathbb{E}[x] = c_1(x) \dfrac{f(x)}{p_1(x)} + c_2(x) \dfrac{f(x)}{p_2(x)},
$$

where $\forall x \in \Omega, c_1(x) + c_2(x) = 1$.

The actual choice of the $c_1(x)$ and $c_2(x)$ can be quite subtle, because the constrain is sooo weak, only $c_1(x) + c_2(x) = 1$. Can we just leverage the PDFs to give a better $c_1(x)$ and $c_2(x)$? Veach presents a method called _power heuristic_, where

$$
  c_1(x) = \dfrac{p_1(x)^2}{p_1(x)^2 + p_2(x)^2}, c_2(x) = \dfrac{p_2(x)^2}{p_1(x)^2 + p_2(x)^2}.
$$

Let's pause and observe the formula. In power heuristic, evaluating $c_1(x)$ requires the evaluation of both $p_1(x)$ and $p_2(x)$ on $x$. For a certain $x$, when $X_1$ gives a higher probability, it weights $X_1$ more on the result, weakening the influence of the other estimator.


In [ ]:
class InverseImportanceSampler(RandomVariable):
    def __init__(self, a: float, b: float):
        self.a_ = a
        self.b_ = b
        self.integral_ = math.log(b) - math.log(a)

    def sample(self) -> float:
        u = np.random.uniform(low=0.0, high=1.0)
        return self.a_ * math.exp(u * self.integral_)

    def pdf(self, x: float) -> float:
        return (1 / x) / self.integral_


print_sampler_report(lambda x: 1 / x, InverseImportanceSampler)
print("-----")
print_sampler_report(lambda x: x**2, InverseImportanceSampler)

As you can see, it gives good estimate on $1/x$ but not $x^2$. Let construct a multiple importance sampler on these two samplers!


In [ ]:
def make_multiple_importance_estimator(integral, rv_factory1, rv_factory2):
    class Estimator(RandomVariable):
        def __init__(self, rv_factory1, rv_factory2):
            self.x1_ = rv_factory1(integral.low_, integral.high_)
            self.x2_ = rv_factory2(integral.low_, integral.high_)

        def sample(self) -> float:
            x1 = self.x1_.sample()
            p11 = self.x1_.pdf(x1)
            p21 = self.x2_.pdf(x1)
            x2 = self.x2_.sample()
            p12 = self.x1_.pdf(x2)
            p22 = self.x2_.pdf(x2)
            c1 = p11**2 / (p11**2 + p21**2)
            c2 = p22**2 / (p12**2 + p22**2)

            return c1 * integral.func_(x1) / p11 + c2 * integral.func_(x2) / p22

    return Estimator(rv_factory1, rv_factory2)


def print_estimator_report(func):
    integral = NumericalIntegral(func, low=0.1, high=1.1)
    mis_estimator = make_multiple_importance_estimator(
        integral, SquareImportanceSampler, InverseImportanceSampler
    )
    uniform_estimator = integral.make_monte_carlo_estimator(Uniform)
    print(f"est_uniform: {uniform_estimator.expect():.4f}")
    print(f"est_mis:     {mis_estimator.expect():.4f}")
    print(
        f"variance_uniform: {make_variance_estimator(uniform_estimator, 10).expect():.4f}"
    )
    print(
        f"variance_mis:     {make_variance_estimator(mis_estimator, 10).expect():.4f}"
    )


print_estimator_report(func=lambda x: x**2)
print("-----")
print_estimator_report(func=lambda x: 1 / x)

## Metropolis-Hastings algorithm

Though this type of method do belongs to MCMC methods before, we also have a unique way to cut into this method.


In [ ]:
def make_metropolis_estimator(func, low, high):
    class Estimator(RandomVariable):
        def __init__(self, a: float, b: float):
            self.a_ = a
            self.b_ = b
            self.func_ = func
            self.sigma_ = 0.3
            self.x_ = low + (high - low) * np.random.random()

        def sample(self):
            y = self.propose(self.x_)
            if self.accept(self.x_, y):
                self.x_ = y
            return self.x_

        def propose(self, x):
            a = (self.a_ - x) / self.sigma_
            b = (self.b_ - x) / self.sigma_
            return truncnorm.rvs(a, b, loc=x, scale=self.sigma_)

        def accept(self, x, y):
            # calculate a(x -> y)
            if y < self.a_ or y >= self.b_:
                return False

            fy = self.func_(y)  # f(y)
            fx = self.func_(x)  # f(x)
            acceptance = min(1.0, fy / fx)

            return np.random.random() <= acceptance

    return Estimator(low, high)


def func(x):
    return norm.pdf(x, loc=0.4, scale=0.1) + 0.2 + norm.pdf(x, loc=0.8, scale=0.05)


estimators = []
for i in range(8):
    estimators.append(make_metropolis_estimator(func=func, low=0.1, high=1.1))
for i in range(2000):
    estimators[i % len(estimators)].sample()  # burn in

inv_integral = 0.0
lst = []
n_samples = 5000
for i in range(n_samples):
    s = estimators[i % len(estimators)].sample()
    inv_integral += 1.0 / func(s)
    lst.append(s)

print(1.0 / (inv_integral / n_samples))

integral = NumericalIntegral(func=func, low=0.1, high=1.1)
est_uniform = integral.make_monte_carlo_estimator(Uniform)
print(integral.get_value())
print(est_uniform.expect(n=n_samples))

In [ ]:
plt.figure(figsize=(10, 6))
counts, bins, patches = plt.hist(lst, bins=200, edgecolor="black")

plt.title("Distribution of Values in Array")
plt.xlabel("Value")
plt.ylabel("Frequency")

x = np.linspace(0.1, 1.1, 1000)
y = np.array([func(xi) for xi in x]) * 16  # chosen scale factor
plt.plot(x, y, "r-")

plt.show()

## Epilogue

We have to admit, the topic of _rendering_ is far from over, and the math is already becoming harder.
But one thing to mention here, so far, the math is _concrete_. We are performing integrals in trivial sets or ranges.
Once you figure out the concept through those (elaborately designed) interfaces, we can proceed to the next level,
to grind through those abstractly defined equations and integrals!
This is where the difficulty for neophyte in rendering comes from.

Please notice, we are entering a brand new mathematically defined world after this lab, not the one that you are familiar with.
We encourage you to ask yourself questions, for example, random variables are defined on _sample space_, so how do we design an estimator on this integral;
what is **dummy variable** in integral; what do we really mean by integrating on solid angle?
To build a comprehensive and mathematically correct path tracer, you should be able to ask and answer these questions.
If you encounter any difficulties, reflect your model of abstraction, and we are happy to discuss these things with you!

There are some good resources that worth reading. One we'd like to recommend here, aside from Physically Based Rendering, is
ROBUST MONTE CARLO METHODS FOR LIGHT TRANSPORT SIMULATION. Check this [meme](https://twitter.com/yiningkarlli/status/1545186689894584320) :)
